# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
experiment = 'DDB_PD_131_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [4]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,Feed Medium #1,Reactor/Plate/Flask Number,Replicate #,Seed,Strain Batch,Initial Culture VolumeNone,Culture Volume Unit,Temperature Setpoint (C),pH Setpoint,pH Control Base Solution,Aeration Gas Type
0,DDB_PD_131_AMBR_DDB35-006_R05_Main_,DDB_BM_016,R05,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R05,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,Air
1,DDB_PD_131_AMBR_DDB35-006_R06_Main_,DDB_BM_016,R06,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R06,2,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
2,DDB_PD_131_AMBR_DDB35-006_R07_Main_,DDB_BM_016,R07,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R07,3,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
3,DDB_PD_131_AMBR_DDB35-006_R08_Main_,DDB_BM_016,R08,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R08,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
4,DDB_PD_131_AMBR_DDB35-006_R09_Main_,DDB_BM_016,R09,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R09,2,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
5,DDB_PD_131_AMBR_DDB35-006_R10_Main_,DDB_BM_016,R10,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R10,3,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
6,DDB_PD_131_AMBR_DDB35-006_R11_Main_,DDB_BM_016,R11,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R11,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
7,DDB_PD_131_AMBR_DDB35-006_R12_Main_,DDB_BM_016,R12,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R12,2,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
8,DDB_PD_131_AMBR_DDB35-006_R13_Main_,DDB_BM_016,R13,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R13,3,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN
9,DDB_PD_131_AMBR_DDB35-006_R14_Main_,DDB_BM_016,R14,AMBR 250,No,Main,DDB_PD_131_AMBR,DDB_FM_016,R14,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,NaN


### 3.1 Online data

In [5]:
online_data_dir = 'online_data_9s_interpolated/'
csv_prefix = '_online_stacked_9s.csv'

In [6]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [8]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,Replicate #,Seed,Strain Batch,Initial Culture VolumeNone,Culture Volume Unit,Temperature Setpoint (C),pH Setpoint,pH Control Base Solution,Aeration Gas Type,Phase
0,0.0000,99.543203,6.509665,999.665160,30.002217,0.352778,100.013705,NaN,20.95,20.689557,...,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,Air,batch
1,0.0025,98.811499,6.511619,998.476214,29.989524,1.304719,100.109567,NaN,20.95,20.689476,...,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,Air,batch
2,0.0050,98.191227,6.513463,999.343662,29.977312,2.307860,100.142303,NaN,20.95,20.689805,...,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,Air,batch
3,0.0075,97.722240,6.512000,1002.933198,29.969866,2.686794,100.036134,NaN,20.95,20.691959,...,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,Air,batch
4,0.0100,97.452985,6.512000,999.217672,29.962434,2.897836,99.987830,NaN,20.95,20.689730,...,1,DDB_PD_131_AMBR_DDB35-006_R17_Pre-Final_,DDB35-006,100,mL,30,6.5,Ammonium hydroxide 8%,Air,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [9]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_131_AMBR_R05__S01,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S01,22.806129,1.0,75.28
1,DDB_PD_131_AMBR_R05__S02,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S02,29.826324,1.5,109.30
2,DDB_PD_131_AMBR_R05__S03,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S03,36.825055,1.5,152.70
3,DDB_PD_131_AMBR_R05__S04,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S04,43.824616,1.5,178.40
4,DDB_PD_131_AMBR_R05__S05,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S05,50.825850,1.5,212.00


#### 3.2.2 Analytical data

In [10]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_131_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,2023-06-12,22.806129,DDB_PD_131_AMBR_R05__S01,acetic acid,0.242199,g/L
1,DDB_PD_131_AMBR_R05__S02_SOA_#1,SOA,Extracellular Region,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,2023-06-12,29.826324,DDB_PD_131_AMBR_R05__S02,acetic acid,0.336886,g/L
2,DDB_PD_131_AMBR_R05__S03_SOA_#1,SOA,Extracellular Region,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,2023-06-12,36.825055,DDB_PD_131_AMBR_R05__S03,acetic acid,0.378179,g/L
3,DDB_PD_131_AMBR_R05__S04_SOA_#1,SOA,Extracellular Region,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,2023-06-12,43.824616,DDB_PD_131_AMBR_R05__S04,acetic acid,0.385366,g/L
4,DDB_PD_131_AMBR_R05__S05_SOA_#1,SOA,Extracellular Region,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,2023-06-12,50.825850,DDB_PD_131_AMBR_R05__S05,acetic acid,0.374385,g/L


##### 3.2.2.1 Pivot analytical data

In [11]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],Timepoint Sample
0,0.0,0.242199,0.153852,0.000000,0.000000,0.000000,11.213361,0.0,0.517768,DDB_PD_131_AMBR_R05__S01
1,0.0,0.336886,0.122987,0.000000,1.148247,0.128030,9.585860,0.0,0.792404,DDB_PD_131_AMBR_R05__S02
2,0.0,0.378179,0.091179,0.183503,1.068548,0.183025,8.430797,0.0,0.862637,DDB_PD_131_AMBR_R05__S03
3,0.0,0.385366,0.048911,0.189592,0.817296,0.193833,7.130182,0.0,0.873692,DDB_PD_131_AMBR_R05__S04
4,0.0,0.374385,0.000000,0.000000,0.618180,0.205275,6.336829,0.0,0.824173,DDB_PD_131_AMBR_R05__S05


#### 3.2.3 Trace_metal_data

In [12]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample


##### 3.2.2.1 Pivot trace metal data

In [13]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,Timepoint Sample


#### 3.2.4 Custom data

In [14]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample


#### 3.2.5 Merging all offline data

In [15]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L]
0,DDB_PD_131_AMBR_R05__S01,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S01,22.806129,1.0,75.28,0.0,0.242199,0.153852,0.000000,0.000000,0.000000,11.213361,0.0,0.517768
1,DDB_PD_131_AMBR_R05__S02,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S02,29.826324,1.5,109.30,0.0,0.336886,0.122987,0.000000,1.148247,0.128030,9.585860,0.0,0.792404
2,DDB_PD_131_AMBR_R05__S03,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S03,36.825055,1.5,152.70,0.0,0.378179,0.091179,0.183503,1.068548,0.183025,8.430797,0.0,0.862637
3,DDB_PD_131_AMBR_R05__S04,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S04,43.824616,1.5,178.40,0.0,0.385366,0.048911,0.189592,0.817296,0.193833,7.130182,0.0,0.873692
4,DDB_PD_131_AMBR_R05__S05,DDB_PD_131_AMBR,DDB_PD_131_AMBR_DDB35-006_R05_Main_,R05,S05,50.825850,1.5,212.00,0.0,0.374385,0.000000,0.000000,0.618180,0.205275,6.336829,0.0,0.824173


## 4. Export data

### 4.2 Separate csv files

In [16]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)